# SHHA Homeowner and Mailing Lists Analysis Notebook
HK, Jan 2026

This notebook provides an analysis of:
1. Addresses
2. "Users" in homeowners database
3. GRIT distribution
4. Email newsletter subscribers
5. Export new email lists


In [1]:
import pandas as pd
import numpy as np
from pathlib import Path

ADDRESSES_FILE = "addresses_export.csv"
USERS_FILE = "export-users-2026-01-18.csv"
SUBSCRIBERS_FILE = "subscribers.csv"

## 1. Analysis of Addresses

In [2]:
addresses_df = pd.read_csv(ADDRESSES_FILE).fillna("")

print("ADDRESS DATA LOADED (ADDRESS VIEW)")
print("----------------------------------")
print(f"Total address records (1 row = 1 physical address): {len(addresses_df):,}")
print()

# ============================================================
# 1) UNIQUE ADDRESSES
# ============================================================

# Each row already represents one physical address
unique_addresses = len(addresses_df)

print("1) UNIQUE ADDRESSES")
print("-------------------")
print(f"Total unique physical addresses: {unique_addresses:,}")
print()

# ============================================================
# 2) HOA MEMBERSHIP STATISTICS
# ============================================================

member_addresses = (addresses_df["Is Member"] == 1).sum()
nonmember_addresses = (addresses_df["Is Member"] == 0).sum()

print("2) HOA MEMBERSHIP STATISTICS")
print("----------------------------")
print(f"Member addresses:     {member_addresses:,}")
print(f"Non-member addresses: {nonmember_addresses:,}")
print(f"Membership rate:      {member_addresses / unique_addresses * 100:.1f}%")
print()

# ============================================================
# 3) HOUSEHOLD COMPOSITION
# ============================================================
# We derive household size from the Users column,
# which is a comma-separated list of users at the address.

def split_users(s: str) -> list:
    if not s:
        return []
    return [u.strip() for u in str(s).split(",") if u.strip()]

addresses_df["user_list"] = addresses_df["Users"].apply(split_users)
addresses_df["household_size"] = addresses_df["user_list"].apply(len)

print("3) HOUSEHOLD COMPOSITION")
print("------------------------")
print(f"Average users per address: {addresses_df['household_size'].mean():.2f}")
print(f"Median users per address:  {addresses_df['household_size'].median():.0f}")
print(f"Maximum users at an address: {addresses_df['household_size'].max():,}")
print()

print("ADDRESS COUNT BY HOUSEHOLD SIZE")
print("-------------------------------")
print(f"Addresses with 0 users:  {(addresses_df['household_size'] == 0).sum():,}")
print(f"Addresses with 1 user:   {(addresses_df['household_size'] == 1).sum():,}")
print(f"Addresses with 2 users:  {(addresses_df['household_size'] == 2).sum():,}")
print(f"Addresses with 3+ users: {(addresses_df['household_size'] >= 3).sum():,}")

# ============================================================
# SEPARATE MAILING ADDRESS ANALYSIS
# ============================================================

def _s(df, col):
    return df[col].fillna("").astype(str).str.strip()

physical_addr = _s(addresses_df, "Address")
mail_addr = _s(addresses_df, "Mail Address")

# Flag addresses with a separate mailing address
addresses_df["has_separate_mailing"] = (
    (mail_addr != "") &
    (mail_addr.str.lower() != physical_addr.str.lower())
)

total_addresses = len(addresses_df)
separate_mailing_count = addresses_df["has_separate_mailing"].sum()

print()
print("SEPARATE MAILING ADDRESS ANALYSIS")
print("---------------------------------")
print(f"Total addresses:                     {total_addresses:,}")
print(f"Addresses with separate mailing:     {separate_mailing_count:,}")
print(f"Percentage with separate mailing:    {separate_mailing_count / total_addresses * 100:.1f}%")

ADDRESS DATA LOADED (ADDRESS VIEW)
----------------------------------
Total address records (1 row = 1 physical address): 2,240

1) UNIQUE ADDRESSES
-------------------
Total unique physical addresses: 2,240

2) HOA MEMBERSHIP STATISTICS
----------------------------
Member addresses:     1,895
Non-member addresses: 345
Membership rate:      84.6%

3) HOUSEHOLD COMPOSITION
------------------------
Average users per address: 1.64
Median users per address:  2
Maximum users at an address: 4

ADDRESS COUNT BY HOUSEHOLD SIZE
-------------------------------
Addresses with 0 users:  9
Addresses with 1 user:   830
Addresses with 2 users:  1,365
Addresses with 3+ users: 36

SEPARATE MAILING ADDRESS ANALYSIS
---------------------------------
Total addresses:                     2,240
Addresses with separate mailing:     436
Percentage with separate mailing:    19.5%


## 2. "Users" in homeowner database

In [21]:
# ============================================================
# USER STATS (USER SYSTEM ONLY)
# Source: export-users-2026-01-18.csv
#
# Key idea:
# - Users.Addresses is a comma-separated string of 0+ addresses.
# - Empty Addresses => user not attached to an address.
# - Some users have multiple addresses.
#
# Contact coverage notes:
# - "Valid email" means non-empty AND not containing 'fake.fake'
# ============================================================

USERS_FILE = "export-users-2026-01-18.csv"
users_df = pd.read_csv(USERS_FILE).fillna("")

def _s(df, col):
    return df[col].fillna("").astype(str).str.strip()

print("USER DATA LOADED (USER SYSTEM)")
print("------------------------------")
print(f"Total user records: {len(users_df):,}")
print()

# ------------------------------------------------------------
# Parse Addresses into counts (0, 1, 2, 3+)
# WARNING: This assumes commas separate multiple addresses.
# If your Addresses strings contain commas inside a single mailing address,
# this will over-count. (Use your 'addresses' view for canonical address work.)
# ------------------------------------------------------------
addresses_raw = _s(users_df, "Addresses")

def split_addresses(s: str) -> list:
    if not s:
        return []
    return [p.strip() for p in s.split(",") if p.strip()]

def count_addresses(s: str) -> int:
    return len(split_addresses(s))

users_df["address_list"] = addresses_raw.apply(split_addresses)
users_df["address_count"] = users_df["address_list"].apply(len)
users_df["has_address"] = users_df["address_count"] > 0

users_with_address = users_df[users_df["has_address"]].copy()

print("1) USERS ASSOCIATED WITH ADDRESSES")
print("----------------------------------")
print(f"Users with 0 addresses: {(users_df['address_count'] == 0).sum():,}")
print(f"Users with 1 address:   {(users_df['address_count'] == 1).sum():,}")
print(f"Users with 2 addresses: {(users_df['address_count'] == 2).sum():,}")
print(f"Users with 3 addresses: {(users_df['address_count'] == 3).sum():,}")
print(f"Users with 4 addresses: {(users_df['address_count'] == 4).sum():,}")
print(f"Users with 5+ addresses:{(users_df['address_count'] >= 5).sum():,}")
print()

# ------------------------------------------------------------
# Contact coverage for users WITH addresses (Venn components)
# ------------------------------------------------------------
email_str = _s(users_with_address, "Email").str.lower()
valid_email = (email_str != "") & (~email_str.str.contains("fake.fake", case=False, na=False))

phone_present = (_s(users_with_address, "Phone") != "") | (_s(users_with_address, "Cell Phone") != "")

email_only = (valid_email & ~phone_present).sum()
phone_only = (~valid_email & phone_present).sum()
email_and_phone = (valid_email & phone_present).sum()
neither = (~valid_email & ~phone_present).sum()

print("2) CONTACT INFORMATION COVERAGE (USERS WITH ADDRESSES)")
print("-----------------------------------------------------")
print("Valid email = non-empty AND does not contain 'fake.fake'\n")
print(f"Email only (valid):   {email_only:,}")
print(f"Phone only:           {phone_only:,}")
print(f"Email + Phone:        {email_and_phone:,}")
print(f"No valid email/phone: {neither:,}")
print(f"TOTAL:                {email_only + phone_only + email_and_phone + neither:,}")
print()

# ============================================================
# ADDRESS-LEVEL CONTACTABILITY (DERIVED FROM USERS)
# Question: for each address, can we reach someone by:
# - email only
# - phone only
# - both
# - neither
# One user or multiple users per address doesn't matter.
# ============================================================

# Flatten user -> address rows
ua = users_with_address.explode("address_list").copy()
ua = ua[ua["address_list"].fillna("").astype(str).str.strip() != ""].copy()
ua = ua.rename(columns={"address_list": "address"})

# Attach contact flags per user-row
ua["valid_email"] = (
    _s(ua, "Email").str.lower().ne("") &
    ~_s(ua, "Email").str.lower().str.contains("fake.fake", na=False)
)
ua["phone_present"] = (_s(ua, "Phone") != "") | (_s(ua, "Cell Phone") != "")

# Aggregate to address-level: reachable if ANY user at that address has it
addr_contact = (
    ua.groupby("address")
      .agg(any_valid_email=("valid_email", "any"),
           any_phone=("phone_present", "any"),
           user_rows=("address", "size"))
      .reset_index()
)

addr_contact["email_only"] = addr_contact["any_valid_email"] & ~addr_contact["any_phone"]
addr_contact["phone_only"] = ~addr_contact["any_valid_email"] & addr_contact["any_phone"]
addr_contact["both"] = addr_contact["any_valid_email"] & addr_contact["any_phone"]
addr_contact["neither"] = ~addr_contact["any_valid_email"] & ~addr_contact["any_phone"]

print("3) ADDRESS-LEVEL CONTACTABILITY (FROM USERS SYSTEM)")
print("---------------------------------------------------")
print("Each address is considered reachable if ANY associated user has the contact method.\n")
print(f"Total distinct addresses (from Users.Addresses): {len(addr_contact):,}")
print()

print("ADDRESS CONTACTABILITY BREAKDOWN")
print("-------------------------------")
print(f"Email only:   {addr_contact['email_only'].sum():,}")
print(f"Phone only:   {addr_contact['phone_only'].sum():,}")
print(f"Both:         {addr_contact['both'].sum():,}")
print(f"Neither:      {addr_contact['neither'].sum():,}")
print(f"TOTAL:        {(addr_contact['email_only'].sum() + addr_contact['phone_only'].sum() + addr_contact['both'].sum() + addr_contact['neither'].sum()):,}")
print()

# Optional: show a few "neither" addresses for inspection
neither_df = addr_contact[addr_contact["neither"]].sort_values("user_rows", ascending=False)
print("SAMPLE ADDRESSES WITH NO EMAIL AND NO PHONE (FOR REVIEW)")
print("--------------------------------------------------------")
print(neither_df[["address", "user_rows"]].head(10))

# ============================================================
# EXPORT: ADDRESSES WITH NO EMAIL AND NO PHONE
# ============================================================

neither_df = addr_contact[addr_contact["neither"]].copy()

export_cols = [
    "address",
    "user_rows",
    "any_valid_email",
    "any_phone",
]

neither_export = neither_df[export_cols].sort_values(
    ["user_rows", "address"], ascending=[False, True]
)

output_path = "addresses_no_email_no_phone.csv"
neither_export.to_csv(output_path, index=False)

print("EXPORT COMPLETE")
print("---------------")
print(f"Addresses with no valid email AND no phone: {len(neither_export):,}")
print(f"Saved to: {output_path}")

USER DATA LOADED (USER SYSTEM)
------------------------------
Total user records: 4,110

1) USERS ASSOCIATED WITH ADDRESSES
----------------------------------
Users with 0 addresses: 592
Users with 1 address:   3,408
Users with 2 addresses: 86
Users with 3 addresses: 9
Users with 4 addresses: 15
Users with 5+ addresses:0

2) CONTACT INFORMATION COVERAGE (USERS WITH ADDRESSES)
-----------------------------------------------------
Valid email = non-empty AND does not contain 'fake.fake'

Email only (valid):   278
Phone only:           587
Email + Phone:        1,999
No valid email/phone: 654
TOTAL:                3,518

3) ADDRESS-LEVEL CONTACTABILITY (FROM USERS SYSTEM)
---------------------------------------------------
Each address is considered reachable if ANY associated user has the contact method.

Total distinct addresses (from Users.Addresses): 2,145

ADDRESS CONTACTABILITY BREAKDOWN
-------------------------------
Email only:   102
Phone only:   171
Both:         1,690
Neither:

### User Roles

In [9]:
# ============================================================
# ROLE BREAKDOWN (ADDRESS USERS EXPORT ONLY)
# Source: address_users_export.csv
# ============================================================

ADDR_USERS_FILE = "address_users_export.csv"
addr_users_df = pd.read_csv(ADDR_USERS_FILE).fillna("")

def _s(df, col):
    return df[col].fillna("").astype(str).str.strip()

print("ADDRESS USERS DATA LOADED")
print("-------------------------")
print(f"Address-user records (people tied to addresses): {len(addr_users_df):,}")
print()

# ------------------------------------------------------------
# Roles (do not normalize; just show what exists)
# ------------------------------------------------------------
print("ROLE VALUES (RAW)")
print("-----------------")
print(addr_users_df["Role"].value_counts(dropna=False))
print()

# ------------------------------------------------------------
# If you want the canonical three-bucket rollup but WITHOUT
# changing raw Role values, map only known roles and flag the rest.
# ------------------------------------------------------------
role_raw = _s(addr_users_df, "Role").str.lower()

is_owner = role_raw == "owner"
is_tenant = role_raw == "tenant"
is_other = role_raw == "other"
is_unknown = ~(is_owner | is_tenant | is_other)

print("ROLE COUNTS (OWNER / TENANT / OTHER)")
print("------------------------------------")
print(f"Owner:   {is_owner.sum():,}")
print(f"Tenant:  {is_tenant.sum():,}")
print(f"Other:   {is_other.sum():,}")
print(f"Unknown: {is_unknown.sum():,}")
print()

if is_unknown.sum() > 0:
    print("UNKNOWN ROLE VALUES (RAW) — PLEASE INSPECT")
    print("-----------------------------------------")
    print(addr_users_df.loc[is_unknown, "Role"].value_counts())

ADDRESS USERS DATA LOADED
-------------------------
Address-user records (people tied to addresses): 3,668

ROLE VALUES (RAW)
-----------------
Role
owner     3603
tenant      57
other        8
Name: count, dtype: int64

ROLE COUNTS (OWNER / TENANT / OTHER)
------------------------------------
Owner:   3,603
Tenant:  57
Other:   8
Unknown: 0



## 3. GRIT Distribution

In [4]:
# ============================================================
# GRIT PRINT MAILING COVERAGE — HOUSEHOLD LEVEL (FINAL SECTION)
#
# GRIT is a PRINT mailing. It is controlled ONLY by the address database:
# - Member households: Is Member == 1
# - A household "gets GRIT" if Mail GRIT == 1 (i.e., not opted out)
# No email/subscribers data is used here.
# ============================================================

import pandas as pd

def _s(df, col):
    return df[col].fillna("").astype(str).str.strip()

# If addresses_df is not already loaded in your notebook, uncomment:
# ADDRESSES_FILE = "addresses_export.csv"  # <-- update filename if needed
# addresses_df = pd.read_csv(ADDRESSES_FILE).fillna("")

print("GRIT PRINT MAILING COVERAGE")
print("---------------------------")
print(f"Total households in address database: {len(addresses_df):,}")
print()

# Some exports store Mail GRIT as 0/1 integers; coerce safely
mail_grit = pd.to_numeric(addresses_df["Mail GRIT"], errors="coerce").fillna(0).astype(int)
is_member = pd.to_numeric(addresses_df["Is Member"], errors="coerce").fillna(0).astype(int)

member_households = addresses_df[is_member == 1]
nonmember_households = addresses_df[is_member == 0]

# GRIT recipients (print) per your definition
gets_grit = mail_grit == 1

member_gets_grit = member_households[gets_grit.loc[member_households.index]]
member_opted_out = member_households[~gets_grit.loc[member_households.index]]

nonmember_gets_grit = nonmember_households[gets_grit.loc[nonmember_households.index]]

print("GRIT PRINT MAILING ('Mail GRIT == 1)")
print("----------------------------------")
print(f"Member households receiving GRIT:     {len(member_gets_grit):,}")
print(f"Member households not receiving GRIT:        {len(member_opted_out):,}")
print(f"Non-member households marked to receive GRIT:  {len(nonmember_gets_grit):,}")
print()

print("COVERAGE RATE (MEMBERS)")
print("-----------------------")
if len(member_households) > 0:
    print(f"Member GRIT coverage: {len(member_gets_grit) / len(member_households) * 100:.1f}%")
else:
    print("Member GRIT coverage: N/A (no member households found)")

GRIT PRINT MAILING COVERAGE
---------------------------
Total households in address database: 2,240

GRIT PRINT MAILING ('Mail GRIT' == 1)
----------------------------------
Member households receiving GRIT:     934
Member households not receiving GRIT:        961
Non-member households marked to receive GRIT:  30

COVERAGE RATE (MEMBERS)
-----------------------
Member GRIT coverage: 49.3%


## 4. Email newsletter subscribers

In [13]:
# ============================================================
# LOAD NEWSLETTER SUBSCRIBERS
# ============================================================

SUBSCRIBERS_FILE = "subscribers.csv"
subscribers_df = pd.read_csv(SUBSCRIBERS_FILE).fillna("")

def _s(df, col):
    return df[col].fillna("").astype(str).str.strip()

print("NEWSLETTER DATA LOADED")
print("----------------------")
print(f"Total subscriber records: {len(subscribers_df):,}")
print()

# ============================================================
# BASIC EMAIL QUALITY FLAGS
# ============================================================

email_str = _s(subscribers_df, "Email")

is_fake_email = email_str.str.contains("fake.fake", case=False, na=False)
has_email = email_str != ""
is_real_email = has_email & ~is_fake_email

# ============================================================
# 1) NEWSLETTERS VS NON-NEWSLETTERS
# ============================================================

in_newsletters = _s(subscribers_df, "Lists").str.contains("Newsletters", case=False, na=False)
not_in_newsletters = ~in_newsletters

print("1) SUBSCRIBER LIST BREAKDOWN")
print("-----------------------------")
print(f"Subscribers in 'Newsletters' list: {in_newsletters.sum():,}")
print(f"Subscribers NOT in 'Newsletters':  {not_in_newsletters.sum():,}")
print()

print("EMAIL QUALITY FOR NON-NEWSLETTER SUBSCRIBERS")
print("--------------------------------------------")
print(f"Real emails:      {(not_in_newsletters & is_real_email).sum():,}")
print(f"Fake.fake emails: {(not_in_newsletters & is_fake_email).sum():,}")
print(f"Blank emails:     {(not_in_newsletters & ~has_email).sum():,}")
print()

# ============================================================
# PREP: BUILD USER LOOKUPS
# (assumes users_df already loaded and has address_count computed)
# ============================================================

users_df["email_norm"] = _s(users_df, "Email").str.lower()
subscribers_df["email_norm"] = email_str.str.lower()

users_df["has_address"] = users_df["address_count"] > 0

user_email_set = set(users_df["email_norm"])
address_user_email_set = set(users_df.loc[users_df["has_address"], "email_norm"])
nonaddress_user_email_set = set(users_df.loc[~users_df["has_address"], "email_norm"])

# ============================================================
# 2) WHO ARE THE NEWSLETTER SUBSCRIBERS?
# ============================================================

newsletter_df = subscribers_df[in_newsletters].copy()

is_user = newsletter_df["email_norm"].isin(user_email_set)
is_address_user = newsletter_df["email_norm"].isin(address_user_email_set)

print("2) WHO RECEIVES NEWSLETTERS?")
print("-----------------------------")
print(f"Total newsletter subscribers: {len(newsletter_df):,}")
print()
print("Breakdown:")
print(f"Homeowners (users with addresses): {is_address_user.sum():,}")
print(f"Other users (no address):          {(is_user & ~is_address_user).sum():,}")
print(f"Not users at all:                  {(~is_user).sum():,}")
print()

# ============================================================
# 3) WHO ARE THE NON-NEWSLETTER SUBSCRIBERS? (REAL EMAILS ONLY)
# ============================================================

non_newsletter_real_df = subscribers_df[
    not_in_newsletters & is_real_email
].copy()

is_user_non = non_newsletter_real_df["email_norm"].isin(user_email_set)
is_address_user_non = non_newsletter_real_df["email_norm"].isin(address_user_email_set)
is_nonaddress_user_non = non_newsletter_real_df["email_norm"].isin(nonaddress_user_email_set)

print("3) WHO ARE THE NON-NEWSLETTER SUBSCRIBERS? (REAL EMAILS ONLY)")
print("------------------------------------------------------------")
print(f"Total non-newsletter subscribers (real emails): {len(non_newsletter_real_df):,}")
print()
print("Breakdown:")
print(f"Users with addresses: {is_address_user_non.sum():,}")
print(f"Users with NO address:{is_nonaddress_user_non.sum():,}")
print(f"Not users at all:     {(~is_user_non).sum():,}")
print()

NEWSLETTER DATA LOADED
----------------------
Total subscriber records: 5,069

1) SUBSCRIBER LIST BREAKDOWN
-----------------------------
Subscribers in 'Newsletters' list: 2,031
Subscribers NOT in 'Newsletters':  3,038

EMAIL QUALITY FOR NON-NEWSLETTER SUBSCRIBERS
--------------------------------------------
Real emails:      1,461
Fake.fake emails: 1,577
Blank emails:     0

2) WHO RECEIVES NEWSLETTERS?
-----------------------------
Total newsletter subscribers: 2,031

Breakdown:
Homeowners (users with addresses): 1,207
Other users (no address):          140
Not users at all:                  684

3) WHO ARE THE NON-NEWSLETTER SUBSCRIBERS? (REAL EMAILS ONLY)
------------------------------------------------------------
Total non-newsletter subscribers (real emails): 1,461

Breakdown:
Users with addresses: 1,099
Users with NO address:156
Not users at all:     206



In [14]:
# ============================================================
# PREP: Normalize emails
# ============================================================

def norm_email(s):
    return s.fillna("").astype(str).str.strip().str.lower()

users_df["email_norm"] = norm_email(users_df["Email"])
subscribers_df["email_norm"] = norm_email(subscribers_df["Email"])

# ------------------------------------------------------------
# Define VALID email (used everywhere below)
# ------------------------------------------------------------
users_df["has_valid_email"] = (
    (users_df["email_norm"] != "") &
    (~users_df["email_norm"].str.contains("fake.fake", case=False, na=False))
)

# Newsletter subscribers (by email)
newsletter_mask = subscribers_df["Lists"].str.contains("Newsletters", case=False, na=False)
newsletter_emails = set(subscribers_df.loc[newsletter_mask, "email_norm"])

# ============================================================
# VENN 1: HOMEOWNERS (USERS WITH ADDRESSES)
# ============================================================

homeowners = users_df[users_df["address_count"] > 0].copy()
homeowners_valid_email = homeowners[homeowners["has_valid_email"]]

homeowner_email_set = set(homeowners_valid_email["email_norm"])

# Venn components
homeowners_valid_email_only = homeowner_email_set - newsletter_emails
homeowners_newsletter_only = newsletter_emails - homeowner_email_set
homeowners_both = homeowner_email_set & newsletter_emails

print("VENN 1: HOMEOWNERS WITH VALID EMAIL vs NEWSLETTER SUBSCRIBERS")
print("------------------------------------------------------------")
print(f"Homeowners with valid email (total): {len(homeowner_email_set):,}")
print(f"Newsletter subscribers (total):      {len(newsletter_emails):,}")
print()
print("Venn regions:")
print(f"Homeowners w/ valid email ONLY (not subscribed): {len(homeowners_valid_email_only):,}")
print(f"Subscribed to newsletters ONLY (not homeowners): {len(homeowners_newsletter_only):,}")
print(f"Intersection (homeowners receiving newsletters): {len(homeowners_both):,}")
print()

print("HOMEOWNER NEWSLETTER COVERAGE (USERS)")
print("------------------------------------")
print(f"Homeowners with valid email: {len(homeowner_email_set):,}")
print(f"Receiving newsletters:      {len(homeowners_both):,}")
print(f"NOT receiving newsletters:  {len(homeowners_valid_email_only):,}")
print(f"Coverage rate:              {len(homeowners_both) / len(homeowner_email_set) * 100:.1f}%")
print()

# ============================================================
# VENN 2: ADDRESSES WITH ≥1 VALID EMAIL vs NEWSLETTER SUBSCRIBERS
# ============================================================

# Build address → user-email mapping from user system
def split_addresses(addr_str):
    if not addr_str:
        return []
    return [a.strip() for a in addr_str.split(",") if a.strip()]

users_df["address_list"] = users_df["Addresses"].fillna("").apply(split_addresses)

user_address_df = users_df.explode("address_list")
user_address_df = user_address_df[user_address_df["address_list"] != ""].copy()
user_address_df = user_address_df.rename(columns={"address_list": "full_address"})

# Addresses with at least one VALID email
addresses_with_valid_email = set(
    user_address_df.loc[user_address_df["has_valid_email"], "full_address"]
)

# Addresses that have ≥1 newsletter subscriber email
addresses_with_newsletter = set(
    user_address_df.loc[
        user_address_df["email_norm"].isin(newsletter_emails),
        "full_address"
    ]
)

# Venn components
addresses_valid_email_only = addresses_with_valid_email - addresses_with_newsletter
addresses_newsletter_only = addresses_with_newsletter - addresses_with_valid_email
addresses_both = addresses_with_valid_email & addresses_with_newsletter

print("VENN 2: ADDRESSES WITH ≥1 VALID EMAIL vs NEWSLETTER COVERAGE")
print("-----------------------------------------------------------")
print(f"Addresses with ≥1 valid email:      {len(addresses_with_valid_email):,}")
print(f"Addresses with newsletter coverage: {len(addresses_with_newsletter):,}")
print()
print("Venn regions:")
print(f"Addresses w/ valid email ONLY (no newsletter): {len(addresses_valid_email_only):,}")
print(f"Addresses w/ newsletter ONLY:                  {len(addresses_newsletter_only):,}")
print(f"Intersection (addresses covered):              {len(addresses_both):,}")
print()

print("ADDRESS-LEVEL NEWSLETTER COVERAGE")
print("---------------------------------")
print(f"Addresses with ≥1 valid email: {len(addresses_with_valid_email):,}")
print(f"Covered by newsletters:       {len(addresses_both):,}")
print(f"NOT covered:                  {len(addresses_valid_email_only):,}")
print(f"Coverage rate:                {len(addresses_both) / len(addresses_with_valid_email) * 100:.1f}%")

VENN 1: HOMEOWNERS WITH VALID EMAIL vs NEWSLETTER SUBSCRIBERS
------------------------------------------------------------
Homeowners with valid email (total): 2,277
Newsletter subscribers (total):      2,031

Venn regions:
Homeowners w/ valid email ONLY (not subscribed): 1,070
Subscribed to newsletters ONLY (not homeowners): 824
Intersection (homeowners receiving newsletters): 1,207

HOMEOWNER NEWSLETTER COVERAGE (USERS)
------------------------------------
Homeowners with valid email: 2,277
Receiving newsletters:      1,207
NOT receiving newsletters:  1,070
Coverage rate:              53.0%

VENN 2: ADDRESSES WITH ≥1 VALID EMAIL vs NEWSLETTER COVERAGE
-----------------------------------------------------------
Addresses with ≥1 valid email:      2,086
Addresses with newsletter coverage: 1,149

Venn regions:
Addresses w/ valid email ONLY (no newsletter): 937
Addresses w/ newsletter ONLY:                  0
Intersection (addresses covered):              1,149

ADDRESS-LEVEL NEWSLETTER 

In [19]:
2031 - 1207

824

## 5. Export New Email Lists

In [22]:
# ============================================================
# FINAL EXPORT: NEWSLETTER SUBSCRIBER LISTS
# ============================================================

import pandas as pd

# -----------------------------
# FILTER OPTION (set this at top)
# -----------------------------
OMIT_REALTORS = True   # If True, exclude any emails that are in the 'Realtors' mailing list

# -----------------------------
# Normalize emails
# -----------------------------
def norm_email(s):
    return s.fillna("").astype(str).str.strip().str.lower()

users_df["email_norm"] = norm_email(users_df["Email"])
subscribers_df["email_norm"] = norm_email(subscribers_df["Email"])

# -----------------------------
# Optional: Realtor filter
# -----------------------------
realtor_email_set = set()
if OMIT_REALTORS:
    in_realtors = subscribers_df["Lists"].astype(str).str.contains("Realtors", case=False, na=False)
    realtor_email_set = set(subscribers_df.loc[in_realtors, "email_norm"])

    print("REALTOR FILTER")
    print("--------------")
    print(f"OMIT_REALTORS = {OMIT_REALTORS}")
    print(f"Emails on Realtors list: {len(realtor_email_set):,}")
    print()

# -----------------------------
# Define valid email + eligible users
# -----------------------------
valid_email_mask = (
    (users_df["email_norm"] != "") &
    (~users_df["email_norm"].str.contains("fake.fake", case=False, na=False))
)

eligible_users = users_df[
    (users_df["address_count"] > 0) &
    valid_email_mask &
    (~users_df["email_norm"].isin(realtor_email_set) if OMIT_REALTORS else True)
].copy()

print("ELIGIBLE USERS (FROM USERS DATABASE)")
print("-----------------------------------")
print(f"Users with addresses and valid email: {len(eligible_users):,}")
if OMIT_REALTORS:
    print(f"  (Realtors omitted): {len(realtor_email_set):,} emails in Realtors list")
print()

# -----------------------------
# Current Newsletter subscribers
# -----------------------------
newsletter_mask = subscribers_df["Lists"].astype(str).str.contains("Newsletters", case=False, na=False)
current_newsletter_emails = set(subscribers_df.loc[newsletter_mask, "email_norm"])

print("CURRENT NEWSLETTER LIST")
print("-----------------------")
print(f"Existing newsletter subscribers: {len(current_newsletter_emails):,}")
print()

# ============================================================
# EXPORT 1: USERS TO ADD (INCREMENTAL)
# ============================================================

to_add = eligible_users[
    ~eligible_users["email_norm"].isin(current_newsletter_emails)
].copy()

add_cols = ["Email", "First Name", "Last Name"]
to_add_export = to_add[add_cols].drop_duplicates().sort_values("Email")

add_path = "newsletter_add_these.csv"
to_add_export.to_csv(add_path, index=False)

print("EXPORT: ADD THESE")
print("-----------------")
print(f"New subscribers to add: {len(to_add_export):,}")
print(f"Saved to: {add_path}")
print()

# ============================================================
# EXPORT 2: COMPLETE AUTHORITATIVE LIST
# ============================================================

complete_export = eligible_users[add_cols].drop_duplicates().sort_values("Email")

complete_path = "newsletter_complete_list.csv"
complete_export.to_csv(complete_path, index=False)

print("EXPORT: COMPLETE LIST")
print("---------------------")
print(f"Total newsletter-eligible users: {len(complete_export):,}")
print(f"Saved to: {complete_path}")
print()

print("DONE")
print("----")
print("Use 'newsletter_add_these.csv' to incrementally add subscribers.")
print("Use 'newsletter_complete_list.csv' to rebuild the list from scratch.")
if OMIT_REALTORS:
    print("Note: Realtors list members were omitted from both exports.")

REALTOR FILTER
--------------
OMIT_REALTORS = True
Emails on Realtors list: 122

ELIGIBLE USERS (FROM USERS DATABASE)
-----------------------------------
Users with addresses and valid email: 2,265
  (Realtors omitted): 122 emails in Realtors list

CURRENT NEWSLETTER LIST
-----------------------
Existing newsletter subscribers: 2,031

EXPORT: ADD THESE
-----------------
New subscribers to add: 1,065
Saved to: newsletter_add_these.csv

EXPORT: COMPLETE LIST
---------------------
Total newsletter-eligible users: 2,265
Saved to: newsletter_complete_list.csv

DONE
----
Use 'newsletter_add_these.csv' to incrementally add subscribers.
Use 'newsletter_complete_list.csv' to rebuild the list from scratch.
Note: Realtors list members were omitted from both exports.
